# tf_to_dict.ipynb
***example of interpreting saved tensorflow model as a python dictionary***
___


## imports

In [ ]:
import tensorflow as tf
import json

from tf_to_dict import tf_to_dict

## tutorial
here we'll load in an example tensorflow model, `pitchfork.h5`, from the `models` directory and use `tf_to_dict` to unpack the important information (like layer weights and biases, activation functions, branching architecture etc.) and save to a python dictionary

In [ ]:
model_name = 'pitchfork'

this model was trained using custom objects, which tensorflow specifically needs to be redefined when we try to load in the saved model - so we'll have to define those here before we can even use the model (don't worry about these, you'll know if you need to define your custom objects properly because tensorflow will complain)

In [ ]:
class InversePCA(tf.keras.layers.Layer):
    """
    Inverse PCA layer for tensorflow neural network
    
    Usage:
        - Define dictionary of custom objects containing Inverse PCA
        - Use arguments of PCA mean and components from PCA of output parameters for inverse PCA (found in JSON dict)
        
    Example:

    > f = open("pcann_info.json")
    >
    > data = json.load(f)
    >
    > pca_comps = np.array(data["pca_comps"])
    > pca_mean = np.array(data["pca_mean"])
    > 
    > custom_objects = {"InversePCA": InversePCA(pca_comps, pca_mean)}
    > pcann_model = tf.keras.models.load_model("pcann_name.h5", custom_objects=custom_objects)
    
    """
    
    def __init__(self, pca_comps, pca_mean, **kwargs):
        super(InversePCA, self).__init__()
        self.pca_comps = pca_comps
        self.pca_mean = pca_mean
        
    def call(self, x):
        y = tf.tensordot(x, np.float32(self.pca_comps),1) + np.float32(self.pca_mean)
        return y
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'pca_comps': self.pca_comps,
            'pca_mean': self.pca_mean
        })
        return config

class WMSE(tf.keras.losses.Loss):
    """
    Weighted Mean Squared Error Loss Function for tensorflow neural network
    
    Usage:
        - Define list of weights with len = labels
        - Use weights as arguments - no need to square, this is handled in-function
        - Typical usage - defining target precision on outputs for the network to achieve, weights parameters in loss calculation to force network to focus on parameters with unc >> weight.
    
    """
    
    def __init__(self, weights, name = "WMSE",**kwargs):
        super(WMSE, self).__init__()
        self.weights = np.float32(weights)
        
    def call(self, y_true, y_pred):
        loss = ((y_true - y_pred)/(self.weights))**2
        return tf.math.reduce_mean(loss)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'weights': self.weights
        })
        return config

def WMSE_metric(y_true, y_pred):
    metric = ((y_true - y_pred)/(weights))**2
    return tf.reduce_mean(metric)


class emulator:
    def __init__(self, emulator_name, file_path='pitchfork/'):
        self.emulator_name = emulator_name
        self.file_path = file_path + self.emulator_name
        
        with open(self.file_path+".pkl", 'rb') as fp:
             self.emulator_dict = pickle.load(fp)

        self.log_inputs_mean = np.array(self.emulator_dict["data_scaling"]["inp_mean"][0])
        
        self.log_inputs_std = np.array(self.emulator_dict["data_scaling"]["inp_std"][0])

        self.log_outputs_mean = np.array(self.emulator_dict["data_scaling"]["classical_out_mean"][0] + self.emulator_dict["data_scaling"]["astero_out_mean"][0])
        
        self.log_outputs_std = np.array(self.emulator_dict["data_scaling"]["classical_out_std"][0] + self.emulator_dict["data_scaling"]["astero_out_std"][0])
            
        self.custom_objects = {"InversePCA": InversePCA(self.emulator_dict['custom_objects']['inverse_pca']['pca_comps'], self.emulator_dict['custom_objects']['inverse_pca']['pca_mean']),"WMSE": WMSE(self.emulator_dict['custom_objects']['WMSE']['weights'])}

        self.model = tf.keras.models.load_model(self.file_path+".h5", custom_objects=self.custom_objects)

        [print(str(key).replace("log_","") + " range: " + "[min = " + str(self.emulator_dict['parameter_ranges'][key]["min"]) + ", max = " + str(self.emulator_dict['parameter_ranges'][key]["max"]) + "]") for key in self.emulator_dict['parameter_ranges'].keys()];

    def predict(self, input_data, n_min=6, n_max=40, verbose=False):
        
        log_inputs = np.log10(input_data)
        
        standardised_log_inputs = (log_inputs - self.log_inputs_mean)/self.log_inputs_std

        standardised_log_outputs = self.model(standardised_log_inputs)

        standardised_log_outputs = np.concatenate((np.array(standardised_log_outputs[0]),np.array(standardised_log_outputs[1])), axis=1)

        log_outputs = (standardised_log_outputs*self.log_outputs_std) + self.log_outputs_mean

        outputs = 10**log_outputs

        outputs[:,2] = log_outputs[:,2] ##we want star_feh in dex

        teff = np.array(((outputs[:,1]*astropy.constants.L_sun) / (4*np.pi*constants.sigma*((outputs[:,0]*astropy.constants.R_sun)**2)))**0.25)
        
        outputs[:,0] = teff
        
        outputs = np.concatenate((np.array(outputs[:,:3]), np.array(outputs[:,n_min-3:n_max-2])), axis=1)

        return outputs


with open(f'models/{model_name}_info.pkl', 'rb') as fp:
    model_info = pickle.load(fp)

custom_objects = {
    "InversePCA": InversePCA(
        model_info['custom_objects']['inverse_pca']['pca_comps'],
        model_info['custom_objects']['inverse_pca']['pca_mean'],
    ),
    "WMSE": WMSE(
        model_info['custom_objects']['WMSE']['weights'],
    ),
}

now we can load in the model using `tf.keras.models.load_model` and pass the custom objects to stop tensorflow from whining (again, you can remove the custom_objects keyword if you don't use any - you'll know if you need them by this point)

In [ ]:
tf_model = tf.keras.models.load_model(
    f'models/{model_name}.h5', # <- change to keras.model if that's how you saved it!
    custom_objects = custom_objects,
)

now we have our model loaded in, we can simply pass this to `tf_to_dict` and wave goodbye to tensorflow!

In [ ]:
wtf_dict = tf_to_dict(tf_model)

this function returns a fully hashable python dictionary of weights, biases, activation functions, layer orders etc.

it also prints out the interpretation of the network structure - you should definitely check this to make sure it matches what you're expecting!

the dictionary is structured in a way that is easily interpreted by our compile functions (see `compile_from_dict.ipynb`), but should retain *some* degree of human readability.

let's take a look:

In [ ]:
print(wtf_dict)

once defined, this wtf_dict can be used directly by the compile_from_dict functions we'll look at in `compile_from_dict`, but it's probably best practice to save it as a json file and then we can load this in in a different environment:

In [ ]:
with open(f'models/{model_name}.json', 'w') as fp:
    json.dump(wtf_dict, fp)

the idea here is that in our tensorflow training environment (where we have tensorflow installed to train models), you'd save a model using tensorflow as usual, and then immediately `tf_to_dict` the saved model and save as a json file to be used in a different environment where you no longer need tensorflow to be installed